In [9]:
import pandas as pd
import numpy as np
import psycopg2

In [10]:
#Creamos una variable para crear la conexión.
conn = psycopg2.connect(
    dbname = "hoteles",
    user = "my_user",
    password = "admin",
    host = "localhost",
    port = "5432"
)

In [11]:
cur = conn.cursor()

In [12]:
cur.execute("SELECT version();")
cur.fetchone()

('PostgreSQL 14.12 (Homebrew) on aarch64-apple-darwin23.4.0, compiled by Apple clang version 15.0.0 (clang-1500.3.9.4), 64-bit',)

### Ciudad


In [6]:
df = pd.read_parquet("../data/info_hoteles_nuestros.parquet")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/info_hoteles_nuestros.parquet'

In [ ]:
tabla_ciudad = df["ciudad"]
tabla_ciudad.nunique()

1

In [ ]:
tabla_ciudad = pd.DataFrame(df["ciudad"].unique(),
                          columns=["nombre_ciudad"])

tabla_ciudad

,nombre_ciudad
0,Madrid


In [ ]:
insert_query = """
        INSERT INTO ciudad (nombre_ciudad)
        VALUES (%s)
"""

In [ ]:
data_to_insert = [(row['nombre_ciudad'],) for _, row in tabla_ciudad.iterrows()]
data_to_insert

[('Madrid',)]

In [ ]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [ ]:
query_comprobacion = """
                        SELECT * FROM ciudad
"""

cur.execute(query_comprobacion)
conn.commit()

In [ ]:
conn.rollback()

### EVENTOS

CREATE TABLE eventos (
    id_evento SERIAL PRIMARY KEY,
    nombre_evento TEXT,
    url_evento TEXT,
    codigo_postal INT,
    direccion TEXT,
    horario TEXT,
    fecha_inicio DATE,
    fecha_fin DATE,
    organizacion TEXT,
    id_ciudad INT REFERENCES 
    ciudad(id_ciudad) ON DELETE CASCADE
);

In [ ]:
tabla_eventos = pd.read_csv("../data/eventos.csv")
tabla_eventos.head()

,nombre_evento,url_evento,codigo_postal,direccion,horario,organizacion,fecha_inicio,fecha_fin,ciudad
0,25º aniversario de la revista La Fragua,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28005.0,CALLE SAN JUSTO 5,NaN,Biblioteca Pública Municipal Iván de Vargas (C...,2025-02-24,2025-03-02,Madrid
1,60 Premio Reina Sofía de Pintura y Escultura,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28009.0,PASEO COLOMBIA 1,NaN,Centro Cultural Casa de Vacas (Retiro),2025-02-27,2025-03-23,Madrid
2,A toda vela,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045.0,PLAZA LEGAZPI 8,NaN,Matadero Madrid,2024-10-01,2025-06-01,Madrid
3,Acompañamiento digital a personas mayores,http://www.madrid.es/sites/v/index.jsp?vgnextc...,NaN,NaN,NaN,NaN,2024-10-14,2025-06-30,NaN
4,Acompañar en la pérdida,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029.0,AVENIDA MONFORTE DE LEMOS 38,17:00,Biblioteca Pública Municipal José Saramago (Fu...,2025-01-14,2025-06-17,Madrid


In [ ]:
#Añadimos las columnas que faltan (diudad_id, id_evento y organicacion)
tabla_eventos["id_ciudad"] = 1
tabla_eventos["organizacion"]
tabla_eventos["fecha_inicio"] = pd.to_datetime(tabla_eventos["fecha_inicio"]).dt.date
tabla_eventos["fecha_fin"] = pd.to_datetime(tabla_eventos["fecha_fin"]).dt.date
tabla_eventos["codigo_postal"] = pd.to_numeric(tabla_eventos["codigo_postal"], errors="coerce").astype("Int64")


In [ ]:
tabla_eventos

,nombre_evento,url_evento,codigo_postal,direccion,horario,fecha_inicio,fecha_fin,organizacion,ciudad,id_ciudad
0,25º aniversario de la revista La Fragua,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28005,CALLE SAN JUSTO 5,NaN,2025-02-24,2025-03-02,Biblioteca Pública Municipal Iván de Vargas (C...,Madrid,1
1,60 Premio Reina Sofía de Pintura y Escultura,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28009,PASEO COLOMBIA 1,NaN,2025-02-27,2025-03-23,Centro Cultural Casa de Vacas (Retiro),Madrid,1
2,A toda vela,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,PLAZA LEGAZPI 8,NaN,2024-10-01,2025-06-01,Matadero Madrid,Madrid,1
3,Acompañamiento digital a personas mayores,http://www.madrid.es/sites/v/index.jsp?vgnextc...,<NA>,NaN,NaN,2024-10-14,2025-06-30,NaN,NaN,1
4,Acompañar en la pérdida,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029,AVENIDA MONFORTE DE LEMOS 38,17:00,2025-01-14,2025-06-17,Biblioteca Pública Municipal José Saramago (Fu...,Madrid,1
...,...,...,...,...,...,...,...,...,...,...
249,"Y tú, ¿cómo viajas?",http://www.madrid.es/sites/v/index.jsp?vgnextc...,28037,CALLE HERMANOS GARCIA NOBLEJAS 14,10:00,2024-12-04,2025-05-28,Biblioteca Pública Municipal Ciudad Lineal (Ci...,Madrid,1
250,Yoga para el buen trato II,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,PASEO CHOPERA 10,18:00,2025-02-18,2025-03-11,Espacio de Igualdad Juana Doña. Arganzuela,Madrid,1
251,'Lina en el mundo de Paul Klee' por Alejandra ...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029,AVENIDA MONFORTE DE LEMOS 38,08:30,2025-02-04,2025-03-25,Biblioteca Pública Municipal José Saramago (Fu...,Madrid,1
252,'Refugiados' por Myriam Laguía,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28019,CALLE ISAAC ALBENIZ 1,19:00,2025-02-14,2025-03-13,Biblioteca Pública Municipal Ana María Matute ...,Madrid,1


In [ ]:
tabla_eventos = tabla_eventos[['nombre_evento', 'url_evento','codigo_postal', 'direccion', 'horario', 'fecha_inicio', 'fecha_fin', 'organizacion', 'id_ciudad']]
tabla_eventos

,nombre_evento,url_evento,codigo_postal,direccion,horario,fecha_inicio,fecha_fin,organizacion,id_ciudad
0,25º aniversario de la revista La Fragua,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28005,CALLE SAN JUSTO 5,NaN,2025-02-24,2025-03-02,Biblioteca Pública Municipal Iván de Vargas (C...,1
1,60 Premio Reina Sofía de Pintura y Escultura,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28009,PASEO COLOMBIA 1,NaN,2025-02-27,2025-03-23,Centro Cultural Casa de Vacas (Retiro),1
2,A toda vela,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,PLAZA LEGAZPI 8,NaN,2024-10-01,2025-06-01,Matadero Madrid,1
3,Acompañamiento digital a personas mayores,http://www.madrid.es/sites/v/index.jsp?vgnextc...,<NA>,NaN,NaN,2024-10-14,2025-06-30,NaN,1
4,Acompañar en la pérdida,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029,AVENIDA MONFORTE DE LEMOS 38,17:00,2025-01-14,2025-06-17,Biblioteca Pública Municipal José Saramago (Fu...,1
...,...,...,...,...,...,...,...,...,...
249,"Y tú, ¿cómo viajas?",http://www.madrid.es/sites/v/index.jsp?vgnextc...,28037,CALLE HERMANOS GARCIA NOBLEJAS 14,10:00,2024-12-04,2025-05-28,Biblioteca Pública Municipal Ciudad Lineal (Ci...,1
250,Yoga para el buen trato II,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045,PASEO CHOPERA 10,18:00,2025-02-18,2025-03-11,Espacio de Igualdad Juana Doña. Arganzuela,1
251,'Lina en el mundo de Paul Klee' por Alejandra ...,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029,AVENIDA MONFORTE DE LEMOS 38,08:30,2025-02-04,2025-03-25,Biblioteca Pública Municipal José Saramago (Fu...,1
252,'Refugiados' por Myriam Laguía,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28019,CALLE ISAAC ALBENIZ 1,19:00,2025-02-14,2025-03-13,Biblioteca Pública Municipal Ana María Matute ...,1


In [ ]:
tabla_eventos["codigo_postal"] = tabla_eventos["codigo_postal"].astype("Int64").astype(object).where(pd.notna(tabla_eventos["codigo_postal"]), None)


In [ ]:
tabla_eventos = tabla_eventos.where(pd.notna(tabla_eventos), None)

In [ ]:
# Ver los tipos de datos en cada columna
print(tabla_eventos.dtypes)

# Verificar si hay valores <NA> en alguna columna
print(tabla_eventos.isna().sum())

# Ver algunas filas con valores nulos
print(tabla_eventos[tabla_eventos.isna().any(axis=1)])

nombre_evento    object
url_evento       object
codigo_postal    object
direccion        object
horario          object
fecha_inicio     object
fecha_fin        object
organizacion     object
id_ciudad         int64
dtype: object
nombre_evento     0
url_evento        0
codigo_postal    30
direccion        30
horario          89
fecha_inicio      0
fecha_fin         0
organizacion     28
id_ciudad         0
dtype: int64
                                         nombre_evento  \
0              25º aniversario de la revista La Fragua   
1         60 Premio Reina Sofía de Pintura y Escultura   
2                                          A toda vela   
3            Acompañamiento digital a personas mayores   
6    Actividades mensuales de los centros de mayore...   
..                                                 ...   
243                     Visitas a Matadero para grupos   
244  Visitas dialogadas. Fuentesal Arenillas e Itzi...   
245                     Visitas guiadas a CentroCentro 

In [ ]:
insert_query = """
        INSERT INTO eventos (nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

In [ ]:
data_to_insert = [
    (
        row["nombre_evento"], 
        row["url_evento"], 
        row["codigo_postal"], 
        row["direccion"], 
        row["horario"], 
        row["fecha_inicio"], 
        row["fecha_fin"], 
        row["organizacion"],
        row["id_ciudad"]
    )
    for _, row in tabla_eventos.iterrows()
]

In [ ]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [ ]:
conn.rollback()

In [ ]:
#Query de comprobación:
query_com = """ 
                    SELECT * FROM eventos
"""
cur.execute(query_com)
cur.fetchmany(3)

[(7,
  '25º aniversario de la revista La Fragua',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=1ea220bed10d4910VgnVCM2000001f4a900aRCRD',
  28005,
  'CALLE SAN JUSTO 5',
  None,
  datetime.date(2025, 2, 24),
  datetime.date(2025, 3, 2),
  'Biblioteca Pública Municipal Iván de Vargas (Centro)',
  1),
 (8,
  '60 Premio Reina Sofía de Pintura y Escultura',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=34c5130393e15910VgnVCM2000001f4a900aRCRD',
  28009,
  'PASEO COLOMBIA 1',
  None,
  datetime.date(2025, 2, 27),
  datetime.date(2025, 3, 23),
  'Centro Cultural Casa de Vacas (Retiro)',
  1),
 (9,
  'A toda vela',
  'http://www.madrid.es/sites/v/index.jsp?vgnextchannel=ca9671ee4a9eb410VgnVCM100000171f5a0aRCRD&vgnextoid=59d5e7da8b822910VgnVCM1000001d4a900aRCRD',
  28045,
  'PLAZA LEGAZPI 8',
  None,
  datetime.date(2024, 10, 1),
  datetime.date(2025, 6, 1),
  'Matadero Ma

### HOTELES

CREATE TABLE hoteles (
    id_hotel SERIAL PRIMARY KEY, -- Puede ser VARCHAR(5)
    nombre_hotel TEXT,
    competencia BOOL,
    valoracion FLOAT CHECK (valoracion BETWEEN 1 AND 5),
    id_ciudad INT REFERENCES ciudad(id_ciudad) ON DELETE CASCADE
);

In [ ]:
# Importamos las tablas de hoteles competencia y nuestros: 
competencia = pd.read_csv("../data/info_competencia.csv")
nuestros = pd.read_csv("../data/info_hoteles_nuestros.csv")

In [ ]:
competencia.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,inicio_estancia,final_estancia,id_hotel,ciudad,id,nombre_hotel,estrellas,precio_noche,fecha_reserva
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,2025-03-01,2025-03-02,113,Madrid,1,ibis budget Madrid Centro Lavapies,4.3,106,2025-02-19
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,2025-03-01,2025-03-02,194,Madrid,2,ibis Styles Madrid Centro Maravillas,4.7,114,2025-02-19
2,4d65096e-5f12-4ce7-9fd6-dd20e3cf5194,443a7307-2183-4051-b0bb-26f12fc1f478,Judith,Rivas,judith.rivas@example.com,True,2025-03-01,2025-03-02,131,Madrid,8,ibis budget Madrid Calle 30,4.4,79,2025-02-19
3,458029ad-cc9b-4d85-8290-fec170bcc02d,e1a34338-a0aa-4b0e-96de-55f884eeb074,Adolfo,Cuervo,adolfo.cuervo@example.com,True,2025-03-01,2025-03-02,114,Madrid,10,ibis Madrid Centro las Ventas,4.5,90,2025-02-19
4,ac0fe86e-90a0-4fe6-a02e-6ea00e9a8621,2949fb90-39ab-4613-b4c1-e238bb56faf3,María Carmen,Navarro,maría carmen.navarro@example.com,True,2025-03-01,2025-03-02,103,Madrid,12,ibis budget Madrid Centro las Ventas,4.3,75,2025-02-19


In [ ]:
nuestros

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,id
0,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid,3
1,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid,4
2,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid,5
3,ccbd9fe9-5a60-4f6f-bed3-73a7158ecba7,df42ae3d-0b9b-4c06-b22e-939323833ef8,Joan,Vazquez,joan.vazquez@example.com,False,2025-02-08,2025-03-01,2025-03-02,36,85.45,Hotel Puerta del Cielo,5.0,Madrid,6
4,99c3dc4f-663c-45f7-849a-ac9313f3746a,a16a9d83-6888-4947-a7ab-ed41203e347c,Chelo,Flor,chelo.flor@example.com,False,2025-02-09,2025-03-01,2025-03-02,43,309.80,Hotel Encanto Real,1.0,Madrid,7
5,a5da0906-0fe5-4c30-b193-863df67a7b84,4b789497-8da4-4962-b0fa-070556648ed1,Dolores,Lobo,dolores.lobo@example.com,False,2025-02-10,2025-03-01,2025-03-02,6,434.88,Palacio del Sol,4.0,Madrid,9
6,3eca014e-f122-41d7-9dd3-b984b9153d57,dbaadb0e-a0ff-436e-9b10-c697fb90b720,Noé,Lobo,noé.lobo@example.com,False,2025-02-02,2025-03-01,2025-03-02,3,216.57,Hotel Jardines del Rey,3.0,Madrid,11
7,8a68fb77-8798-4902-a92d-642ea5e47083,26ca67cd-3762-4f90-9372-c5f3d49df733,Omar,Lamas,omar.lamas@example.com,False,2025-02-10,2025-03-01,2025-03-02,29,378.10,Hotel Las Estrellas,4.0,Madrid,14
8,5e3de36b-3d2f-4cef-98d3-df826c56a4b1,a263c6e8-96af-4720-96f0-04cfe51116c6,Ágata,Pinedo,ágata.pinedo@example.com,False,2025-02-01,2025-03-01,2025-03-02,47,358.07,Gran Hotel Madrid,2.0,Madrid,16
9,96a3727c-99fb-4ae0-a6c3-886504499ec2,7164885d-7481-40cc-9aa7-025264d00798,Javier,Mancebo,javier.mancebo@example.com,False,2025-02-12,2025-03-01,2025-03-02,27,228.02,Hotel Palacio Imperial,2.0,Madrid,19


In [ ]:
#Unimos las dos columnas en una sola tabla: 
tabla_hoteles = pd.concat([competencia, nuestros], axis=0)
tabla_hoteles

,id_reserva,id_cliente,nombre,apellido,mail,competencia,inicio_estancia,final_estancia,id_hotel,ciudad,id,nombre_hotel,estrellas,precio_noche,fecha_reserva
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,2025-03-01,2025-03-02,113,Madrid,1,ibis budget Madrid Centro Lavapies,4.3,106.00,2025-02-19
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,2025-03-01,2025-03-02,194,Madrid,2,ibis Styles Madrid Centro Maravillas,4.7,114.00,2025-02-19
2,4d65096e-5f12-4ce7-9fd6-dd20e3cf5194,443a7307-2183-4051-b0bb-26f12fc1f478,Judith,Rivas,judith.rivas@example.com,True,2025-03-01,2025-03-02,131,Madrid,8,ibis budget Madrid Calle 30,4.4,79.00,2025-02-19
3,458029ad-cc9b-4d85-8290-fec170bcc02d,e1a34338-a0aa-4b0e-96de-55f884eeb074,Adolfo,Cuervo,adolfo.cuervo@example.com,True,2025-03-01,2025-03-02,114,Madrid,10,ibis Madrid Centro las Ventas,4.5,90.00,2025-02-19
4,ac0fe86e-90a0-4fe6-a02e-6ea00e9a8621,2949fb90-39ab-4613-b4c1-e238bb56faf3,María Carmen,Navarro,maría carmen.navarro@example.com,True,2025-03-01,2025-03-02,103,Madrid,12,ibis budget Madrid Centro las Ventas,4.3,75.00,2025-02-19
5,8d2d3ea2-14bb-4762-a7a4-a44e75d2d2f4,6c104070-aad0-4372-a6dc-cd38a6f7c3cf,Ricardo,Caballero,ricardo.caballero@example.com,True,2025-03-01,2025-03-02,181,Madrid,13,ibis Styles Madrid City Las Ventas,4.6,98.00,2025-02-19
6,f4cf7cbf-7a2a-4e44-9b9d-e2c9a444ec78,b1826b3e-e9aa-4a40-b94a-09ad8e7d6507,Lázaro,Pardo,lázaro.pardo@example.com,True,2025-03-01,2025-03-02,128,Madrid,17,ibis budget Madrid Calle Alcalá,4.3,71.00,2025-02-19
7,4e4f9542-3b51-4da1-b75e-b8c8a2018b6e,85306c06-a7f1-4fdb-807e-66b2178e93c4,Angelina,Priego,angelina.priego@example.com,True,2025-03-01,2025-03-02,186,Madrid,32,ibis Madrid Calle Alcalá,4.5,78.00,2025-02-19
8,7acd5eae-e278-4568-9e35-638532e10b87,43ffdcc5-51b6-4eda-81f5-df2c1ed1adbe,María Belén,Escalona,maría belén.escalona@example.com,True,2025-03-01,2025-03-02,135,Madrid,47,ibis budget Madrid Vallecas,4.3,70.00,2025-02-19
9,946c7d6c-9523-4056-bc05-4f25b7582202,de5abee1-33ee-421f-9ebd-1baaafafccf9,Rosalva,Cabañas,rosalva.cabañas@example.com,True,2025-03-01,2025-03-02,117,Madrid,79,ibis Madrid Getafe,4.4,64.00,2025-02-19


In [ ]:
#Cambiamos el nombre de la columan estrellas por valoracion:
tabla_hoteles = tabla_hoteles.rename(columns={"estrellas": "valoracion"})
tabla_hoteles

,id_reserva,id_cliente,nombre,apellido,mail,competencia,inicio_estancia,final_estancia,id_hotel,ciudad,id,nombre_hotel,valoracion,precio_noche,fecha_reserva
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,2025-03-01,2025-03-02,113,Madrid,1,ibis budget Madrid Centro Lavapies,4.3,106.00,2025-02-19
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,2025-03-01,2025-03-02,194,Madrid,2,ibis Styles Madrid Centro Maravillas,4.7,114.00,2025-02-19
2,4d65096e-5f12-4ce7-9fd6-dd20e3cf5194,443a7307-2183-4051-b0bb-26f12fc1f478,Judith,Rivas,judith.rivas@example.com,True,2025-03-01,2025-03-02,131,Madrid,8,ibis budget Madrid Calle 30,4.4,79.00,2025-02-19
3,458029ad-cc9b-4d85-8290-fec170bcc02d,e1a34338-a0aa-4b0e-96de-55f884eeb074,Adolfo,Cuervo,adolfo.cuervo@example.com,True,2025-03-01,2025-03-02,114,Madrid,10,ibis Madrid Centro las Ventas,4.5,90.00,2025-02-19
4,ac0fe86e-90a0-4fe6-a02e-6ea00e9a8621,2949fb90-39ab-4613-b4c1-e238bb56faf3,María Carmen,Navarro,maría carmen.navarro@example.com,True,2025-03-01,2025-03-02,103,Madrid,12,ibis budget Madrid Centro las Ventas,4.3,75.00,2025-02-19
5,8d2d3ea2-14bb-4762-a7a4-a44e75d2d2f4,6c104070-aad0-4372-a6dc-cd38a6f7c3cf,Ricardo,Caballero,ricardo.caballero@example.com,True,2025-03-01,2025-03-02,181,Madrid,13,ibis Styles Madrid City Las Ventas,4.6,98.00,2025-02-19
6,f4cf7cbf-7a2a-4e44-9b9d-e2c9a444ec78,b1826b3e-e9aa-4a40-b94a-09ad8e7d6507,Lázaro,Pardo,lázaro.pardo@example.com,True,2025-03-01,2025-03-02,128,Madrid,17,ibis budget Madrid Calle Alcalá,4.3,71.00,2025-02-19
7,4e4f9542-3b51-4da1-b75e-b8c8a2018b6e,85306c06-a7f1-4fdb-807e-66b2178e93c4,Angelina,Priego,angelina.priego@example.com,True,2025-03-01,2025-03-02,186,Madrid,32,ibis Madrid Calle Alcalá,4.5,78.00,2025-02-19
8,7acd5eae-e278-4568-9e35-638532e10b87,43ffdcc5-51b6-4eda-81f5-df2c1ed1adbe,María Belén,Escalona,maría belén.escalona@example.com,True,2025-03-01,2025-03-02,135,Madrid,47,ibis budget Madrid Vallecas,4.3,70.00,2025-02-19
9,946c7d6c-9523-4056-bc05-4f25b7582202,de5abee1-33ee-421f-9ebd-1baaafafccf9,Rosalva,Cabañas,rosalva.cabañas@example.com,True,2025-03-01,2025-03-02,117,Madrid,79,ibis Madrid Getafe,4.4,64.00,2025-02-19


In [ ]:
tabla_hoteles["id_ciudad"] = 1

In [ ]:
   id_hotel SERIAL PRIMARY KEY, -- Puede ser VARCHAR(5)
    nombre_hotel TEXT,
    competencia BOOL,
    valoracion FLOAT CHECK (valoracion BETWEEN 1 AND 5),
    id_ciudad INT REFERENCES ciudad(id_ciudad) ON DELETE CASCADE

In [ ]:
tabla_hoteles = tabla_hoteles[["id_hotel", "nombre_hotel", "competencia", "valoracion", "id_ciudad"]]
tabla_hoteles

,id_hotel,nombre_hotel,competencia,valoracion,id_ciudad
0,113,ibis budget Madrid Centro Lavapies,True,4.3,1
1,194,ibis Styles Madrid Centro Maravillas,True,4.7,1
2,131,ibis budget Madrid Calle 30,True,4.4,1
3,114,ibis Madrid Centro las Ventas,True,4.5,1
4,103,ibis budget Madrid Centro las Ventas,True,4.3,1
5,181,ibis Styles Madrid City Las Ventas,True,4.6,1
6,128,ibis budget Madrid Calle Alcalá,True,4.3,1
7,186,ibis Madrid Calle Alcalá,True,4.5,1
8,135,ibis budget Madrid Vallecas,True,4.3,1
9,117,ibis Madrid Getafe,True,4.4,1


In [ ]:
#añadimos una columna id_hotel donde añadimos un seriado de 1 a n:
tabla_hoteles["id_hotel"] = range(1, 1+len(tabla_hoteles))
tabla_hoteles

/var/folders/_s/kc_rj_092690fgthyqft8hk40000gn/T/ipykernel_2885/1753282041.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabla_hoteles["id_hotel"] = range(1, 1+len(tabla_hoteles))


,id_hotel,nombre_hotel,competencia,valoracion,id_ciudad
0,1,ibis budget Madrid Centro Lavapies,True,4.3,1
1,2,ibis Styles Madrid Centro Maravillas,True,4.7,1
2,3,ibis budget Madrid Calle 30,True,4.4,1
3,4,ibis Madrid Centro las Ventas,True,4.5,1
4,5,ibis budget Madrid Centro las Ventas,True,4.3,1
5,6,ibis Styles Madrid City Las Ventas,True,4.6,1
6,7,ibis budget Madrid Calle Alcalá,True,4.3,1
7,8,ibis Madrid Calle Alcalá,True,4.5,1
8,9,ibis budget Madrid Vallecas,True,4.3,1
9,10,ibis Madrid Getafe,True,4.4,1


In [ ]:
data_to_insert = [
    (
        row["id_hotel"], 
        row["nombre_hotel"], 
        row["competencia"], 
        row["valoracion"],
        row["id_ciudad"]
    )
    for _, row in tabla_hoteles.iterrows()
]

In [ ]:
#añadimos la columna id_ciudad a la base de datos: 
data_to_insert = [
    (
        row["id_ciudad"]
    )
    for _, row in tabla_hoteles.iterrows()
]

In [ ]:
insert_query = """  
        INSERT INTO hoteles (id_ciudad)
        VALUES (%s)
"""

In [ ]:
insert_query = """
        INSERT INTO hoteles (id_hotel, nombre_hotel, competencia, valoracion, id_ciudad)
        VALUES (%s, %s, %s, %s, %s)
"""

In [ ]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [ ]:

conn.rollback()

### CLIENTES

In [ ]:
CREATE TABLE clientes (
    id_cliente VARCHAR(50) PRIMARY KEY,
    nombre TEXT,
    apellido TEXT,
    mail TEXT UNIQUE CHECK (mail LIKE '%@%')
);

In [ ]:
clientes = pd.read_csv("../data/limpieza_reservas_hoteles.csv")

In [ ]:
clientes.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,id
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,2025-02-19,2025-03-01,2025-03-02,113,NaN,NaN,NaN,NaN,1
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,2025-02-19,2025-03-01,2025-03-02,194,NaN,NaN,NaN,NaN,2
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid,3
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid,4
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid,5


In [ ]:
tabla_clientes = clientes[["id_cliente", "nombre", "apellido", "mail"]]
tabla_clientes

,id_cliente,nombre,apellido,mail
0,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com
1,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com
2,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com
3,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com
4,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com
...,...,...,...,...
15093,cc50c0c1-a589-44b7-8a41-2c731d033f89,Victoriano,Miralles,victoriano.miralles@example.com
15094,2bf0215f-7f7b-4cfb-9a37-d2022c492410,Anacleto,Toledo,anacleto.toledo@example.com
15095,4ba34638-c43e-455f-886d-b3e3dd223a07,Máxima,Mayoral,máxima.mayoral@example.com
15096,a017cb9c-160b-4b90-b4a9-4bdb2dce546a,Rafa,Alcaraz,rafa.alcaraz@example.com


In [ ]:
#Borramos duplicados
tabla_clientes = tabla_clientes.drop_duplicates()
tabla_clientes

,id_cliente,nombre,apellido,mail
0,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com
1,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com
2,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com
3,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com
4,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com
...,...,...,...,...
14995,de729428-ec23-4665-a5c9-79e3ec282470,Lorena,Román,lorena.román@example.com
14996,8469b819-d96a-4904-9cdd-710764f6301f,Toño,Narváez,toño.narváez@example.com
14997,e89751f4-3621-4636-81d3-90e2baa709cb,Nadia,Verdugo,nadia.verdugo@example.com
14998,c4117358-fd89-44cd-b388-4ec6d3538c72,Custodia,Marco,custodia.marco@example.com


In [ ]:
#Rehacemos los índices de id_cliente con un seriado de 1 a n:
tabla_clientes["id_cliente"] = range(1, 1+len(tabla_clientes))
tabla_clientes


/var/folders/_s/kc_rj_092690fgthyqft8hk40000gn/T/ipykernel_2885/3063268258.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tabla_clientes["id_cliente"] = range(1, 1+len(tabla_clientes))


,id_cliente,nombre,apellido,mail
0,1,Feliciana,Cantón,feliciana.cantón@example.com
1,2,Leonardo,Hierro,leonardo.hierro@example.com
2,3,Maite,Calatayud,maite.calatayud@example.com
3,4,Tecla,Bonet,tecla.bonet@example.com
4,5,Amílcar,Andrés,amílcar.andrés@example.com
...,...,...,...,...
14995,14996,Lorena,Román,lorena.román@example.com
14996,14997,Toño,Narváez,toño.narváez@example.com
14997,14998,Nadia,Verdugo,nadia.verdugo@example.com
14998,14999,Custodia,Marco,custodia.marco@example.com


In [ ]:
CREATE TABLE clientes (
    id_cliente VARCHAR(50) PRIMARY KEY,
    nombre TEXT,
    apellido TEXT,
    mail TEXT UNIQUE CHECK (mail LIKE '%@%')
);

In [ ]:
data_to_insert = [
    (
        row["id_cliente"], 
        row["nombre"], 
        row["apellido"], 
        row["mail"]
    )
    for _, row in tabla_clientes.iterrows()
]

In [ ]:
insert_query = """  
        INSERT INTO clientes (id_cliente, nombre, apellido, mail)
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (mail) DO NOTHING;
"""

In [ ]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [ ]:
conn.rollback()

### RESERVAS

In [7]:
CREATE TABLE reservas (
    id_reserva VARCHAR(50) PRIMARY KEY,
    fecha_reserva DATE,
    inicio_estancia DATE,
    final_estancia DATE,
    precio_noche FLOAT CHECK (precio_noche >= 0),
    id_cliente VARCHAR(50) REFERENCES clientes(id_cliente) ON DELETE CASCADE,
    id_hotel INT REFERENCES hoteles(id_hotel) ON DELETE CASCADE
    
);

SyntaxError: invalid syntax (3127217797.py, line 1)

In [40]:
reservas = pd.read_csv("../data/limpieza_reservas_hoteles.csv")
reservas.head()

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,precio_noche,nombre_hotel,valoracion,ciudad,id
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,2025-02-19,2025-03-01,2025-03-02,NaN,NaN,NaN,NaN,1
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,2025-02-19,2025-03-01,2025-03-02,NaN,NaN,NaN,NaN,2
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,119.96,Hotel Monte Verde,1.0,Madrid,3
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,211.53,Hotel Brisas del Mar,5.0,Madrid,4
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,378.37,Hotel Camino del Sol,1.0,Madrid,5


In [45]:
#Eliminamos la columna id_cliente
reservas = reservas.drop(columns=["id_cliente"])
reservas

,id_reserva,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,precio_noche,nombre_hotel,valoracion,ciudad,id
0,5256cc90-139b-43d2-8ec5-412495d751cf,Feliciana,Cantón,feliciana.cantón@example.com,True,2025-02-19,2025-03-01,2025-03-02,NaN,NaN,NaN,NaN,1
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,Leonardo,Hierro,leonardo.hierro@example.com,True,2025-02-19,2025-03-01,2025-03-02,NaN,NaN,NaN,NaN,2
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,119.96,Hotel Monte Verde,1.0,Madrid,3
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,211.53,Hotel Brisas del Mar,5.0,Madrid,4
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,378.37,Hotel Camino del Sol,1.0,Madrid,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15093,49c9a352-a260-46fb-9edc-a2c2d05fdfb9,Victoriano,Miralles,victoriano.miralles@example.com,False,2025-02-02,2025-03-01,2025-03-02,367.65,Hotel Las Estrellas,3.0,Madrid,3320
15094,f07370e3-feeb-4893-a9ff-4ec4fb518a55,Anacleto,Toledo,anacleto.toledo@example.com,False,2025-02-07,2025-03-01,2025-03-02,144.35,Gran Hotel Madrid,4.0,Madrid,12211
15095,1d4b9293-1733-45ce-8a8e-79d3ac4694eb,Máxima,Mayoral,máxima.mayoral@example.com,False,2025-02-08,2025-03-01,2025-03-02,463.32,Hotel Jardines del Rey,2.0,Madrid,7155
15096,d6823ca7-e236-48b1-bf68-6b88d9b6534a,Rafa,Alcaraz,rafa.alcaraz@example.com,False,2025-02-10,2025-03-01,2025-03-02,247.63,Hotel Palacio Imperial,1.0,Madrid,13302


In [46]:
#cambiamos el nombre de id por id_cliente:
reservas = reservas.rename(columns={"id": "id_cliente"})

In [66]:
tabla_reservas = reservas[["id_reserva","fecha_reserva","inicio_estancia","final_estancia","precio_noche"]]

In [67]:
tabla_reservas

,id_reserva,fecha_reserva,inicio_estancia,final_estancia,precio_noche
0,5256cc90-139b-43d2-8ec5-412495d751cf,2025-02-19,2025-03-01,2025-03-02,NaN
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,2025-02-19,2025-03-01,2025-03-02,NaN
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,2025-02-09,2025-03-01,2025-03-02,119.96
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,2025-02-08,2025-03-01,2025-03-02,211.53
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,2025-02-02,2025-03-01,2025-03-02,378.37
...,...,...,...,...,...
15093,49c9a352-a260-46fb-9edc-a2c2d05fdfb9,2025-02-02,2025-03-01,2025-03-02,367.65
15094,f07370e3-feeb-4893-a9ff-4ec4fb518a55,2025-02-07,2025-03-01,2025-03-02,144.35
15095,1d4b9293-1733-45ce-8a8e-79d3ac4694eb,2025-02-08,2025-03-01,2025-03-02,463.32
15096,d6823ca7-e236-48b1-bf68-6b88d9b6534a,2025-02-10,2025-03-01,2025-03-02,247.63


In [72]:
#Eliminamos duplicados en la tabla:
tabla_reservas = tabla_reservas.drop_duplicates()
tabla_reservas

,id_reserva,fecha_reserva,inicio_estancia,final_estancia,precio_noche
0,5256cc90-139b-43d2-8ec5-412495d751cf,2025-02-19,2025-03-01,2025-03-02,NaN
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,2025-02-19,2025-03-01,2025-03-02,NaN
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,2025-02-09,2025-03-01,2025-03-02,119.96
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,2025-02-08,2025-03-01,2025-03-02,211.53
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,2025-02-02,2025-03-01,2025-03-02,378.37
...,...,...,...,...,...
14995,58acd6f3-7160-4973-b951-b14188937444,2025-02-06,2025-03-01,2025-03-02,111.78
14996,aa092c3e-7fe6-4985-a5a8-cacd38554b1b,2025-02-19,2025-03-01,2025-03-02,NaN
14997,af0f6afa-e7d2-410a-8cdd-5eb62b1e4788,2025-02-11,2025-03-01,2025-03-02,442.96
14998,6d1f590c-ae87-4582-bc99-9b8cbf1d8e28,2025-02-19,2025-03-01,2025-03-02,NaN


In [73]:
# Crear lista de datos a insertar
data_to_insert = [
    (
        row["id_reserva"], 
        row["fecha_reserva"], 
        row["inicio_estancia"], 
        row["final_estancia"],
        row["precio_noche"],
    )
    for _ , row in tabla_reservas.iterrows()
]

In [74]:
insert_query = """
    INSERT INTO reservas (id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche)
    VALUES (%s, %s, %s, %s, %s)
"""

In [63]:
tabla_reservas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_reserva       15098 non-null  object 
 1   fecha_reserva    15098 non-null  object 
 2   inicio_estancia  15098 non-null  object 
 3   final_estancia   15098 non-null  object 
 4   precio_noche     9874 non-null   float64
 5   id_cliente       15098 non-null  int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 707.8+ KB


In [76]:
cur.executemany(insert_query, data_to_insert)
conn.commit()

In [75]:
conn.rollback()